In [9]:
import torch, numpy
from torch.func import hessian
from torch.func import jacrev
# I am going to use the optimzer from torch.
# pip install pytorch-minimize
from torchmin import minimize

def f(x):
    """ Gen Rosenbrock function.
    Args: x (torch.Tensor): Input tensor of shape (n).
    Returns: torch.Tensor: Output tensor of shape (1).
    """
    return torch.sum(100.0 * (x[1:] - x[:-1]**2)**2 + (1 - x[:-1])**2, dim=-1)

def dfSQ(x):
    """ Sum of squares of gradient for f
    Args: x: torch tensor of length n.
    Returns: torch scalar.
    """
    # Computes reverse mode gradient using PyTorch
    g=jacrev(f)(x)
    g=torch.square(g)
    return torch.sum(g)

In [34]:
# Define a random IC
n = 10
x = (torch.randn(n)).numpy()
# Minimze sum of squares
xMin=minimize(dfSQ, x, method='bfgs')
print(xMin)

  message: A non-descent direction was encountered.
  success: False
   status: 4
      fun: tensor(3.6244e-11)
        x: tensor([-5.5538e-01,  3.2245e-01,  1.1518e-01,  2.3506e-02,  1.0661e-02,
                    1.0218e-02,  1.0209e-02,  1.0204e-02,  1.0004e-02,  1.0009e-04])
      nit: 124
     grad: tensor([ 1.8709e-03,  1.7725e-03,  3.5906e-05, -7.5862e-04, -2.4379e-04,
                   -3.9892e-04,  1.3085e-03,  1.0256e-03, -7.0519e-04,  4.7669e-04])
     nfev: 127
 hess_inv: tensor([[ 2.1846e-02, -2.4166e-02, -1.5509e-02, -3.5534e-03, -1.5858e-04,
                     1.5979e-05,  3.6914e-06,  2.7336e-06, -9.4636e-06,  2.9116e-06],
                   [-2.4167e-02,  2.6741e-02,  1.7164e-02,  3.9338e-03,  1.7547e-04,
                    -1.8172e-05, -3.8290e-06, -2.6956e-06,  1.0513e-05, -3.1253e-06],
                   [-1.5509e-02,  1.7164e-02,  1.1027e-02,  2.5298e-03,  1.1283e-04,
                    -1.1899e-05, -2.3456e-06, -1.8206e-06,  6.8081e-06, -2.0084e-06],
       

In [32]:
# Convert CP to Torch to compute hessian 
g = (jacrev)(f)(xMin.x)
hess=hessian(f)(xMin.x)
GradRes = torch.linalg.norm(g)
# Compute eigenvalues
Lambdas=torch.linalg.eigvalsh(hess)
AppInvLambdas=torch.linalg.eigvalsh(xMin.hess_inv)
# sort and compare eigenvalues
torch.sort(Lambdas)
AppLambdas = torch.reciprocal(AppInvLambdas)
torch.sort(AppLambdas)
[AppLambdas, Lambdas]

[tensor([44501532.0000,   364637.7188,   111738.1797,    90431.2969,
            82056.5391,    80759.9844,    78423.4531,    73472.7188,
            65632.9922,    61932.9492]),
 tensor([-1.5903e-01,  1.7644e+02,  1.9115e+02,  1.9427e+02,  1.9735e+02,
          2.0035e+02,  2.0403e+02,  2.0699e+02,  2.1716e+02,  4.1347e+02])]